# Problem Definition

Many people struggle to get loans due to insufficient or non-existent credit histories. And, unfortunately, this population is often taken advantage of by untrustworthy lenders.

- Home Credit Group

Home Credit strives to broaden financial inclusion for the unbanked population by providing a positive and safe borrowing experience. In order to make sure this underserved population has a positive loan experience, Home Credit makes use of a variety of alternative data--including telco and transactional information--to predict their clients' repayment abilities.

While Home Credit is currently using various statistical and machine learning methods to make these predictions, they're challenging Kagglers to help them unlock the full potential of their data. Doing so will ensure that clients capable of repayment are not rejected and that loans are given with a principal, maturity, and repayment calendar that will empower their clients to be successful.

### Data Description

See https://www.kaggle.com/c/home-credit-default-risk/data

There are 7 different sources of data:

* <b>application_train.csv</b>: contains the main training data with information about each loan application at Home Credit. Every loan has its own row and is identified by the feature `SK_ID_CURR`. The training application data comes with the `TARGET` indicating 0 (the loan was repaid) or 1 (the loan was not repaid). 
* <b>application_test.csv</b>: contains the main testing data with information about each loan application at Home Credit. Every loan has its own row and is identified by the feature `SK_ID_CURR`. 

#### Imports

In [28]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import OneHotEncoder, Imputer, PolynomialFeatures, MinMaxScaler
from sklearn.linear_model import LogisticRegression

# Data

#### Main train data

In [2]:
def get_train_data(path: str, target_name: str) -> tuple:
    df = pd.read_csv(path)
    target = df[target_name]    
    features = df.drop(target_name, axis=1)
    return features, target

In [3]:
path = 'data/application_train.csv'
target_name = 'TARGET'
train_features, train_target = get_train_data(path, target_name)

In [4]:
train_features.shape, train_target.shape

((307511, 121), (307511,))

In [5]:
train_features.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
train_target.head()

0    1
1    0
2    0
3    0
4    0
Name: TARGET, dtype: int64

#### Test data

In [7]:
test_features = pd.read_csv("data/application_test.csv")
test_features.shape

(48744, 121)

In [36]:
test_features.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


#### Dealing with categorical values

Convert categorical features to one-hot encoded features. This is due to the fact that implementations of most machine learning models cannot directly deal with categorical data. We will use pandas native one hot encoder. Howerver, to align train and test data, we will simply remove the additional columns, since they reflect only a small portion of data.

In [8]:
def make_one_hot_encoded(train_features: pd.DataFrame, test_features: pd.DataFrame) -> tuple:
    train_1h = pd.get_dummies(train_features)
    test_1h = pd.get_dummies(test_features)
    return train_1h.align(test_1h, join='inner', axis=1)

In [9]:
train_1h, test_1h = make_one_hot_encoded(train_features, test_features)
assert train_1h.shape[1]==test_1h.shape[1]

In [10]:
train_1h.shape, test_1h.shape

((307511, 242), (48744, 242))

Lets look at the one hot encoded colum s

In [11]:
def get_extra_columns(before: np.ndarray, after: np.ndarray) -> list:
    return list(set(after).difference(set(before)))

In [12]:
train_1h[get_extra_columns(train_features.columns.values, train_1h.columns.values)].head()

,ORGANIZATION_TYPE_Business Entity Type 2,ORGANIZATION_TYPE_Telecom,ORGANIZATION_TYPE_Industry: type 2,OCCUPATION_TYPE_HR staff,WEEKDAY_APPR_PROCESS_START_TUESDAY,NAME_FAMILY_STATUS_Single / not married,ORGANIZATION_TYPE_Trade: type 3,ORGANIZATION_TYPE_Trade: type 1,OCCUPATION_TYPE_Sales staff,NAME_TYPE_SUITE_Unaccompanied,...,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,ORGANIZATION_TYPE_Business Entity Type 1,ORGANIZATION_TYPE_XNA,ORGANIZATION_TYPE_Industry: type 11,ORGANIZATION_TYPE_Cleaning,FLAG_OWN_REALTY_N,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_Realtor,FONDKAPREMONT_MODE_reg oper account
0,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,1,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
2,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [13]:
test_1h[get_extra_columns(test_features.columns.values, test_1h.columns.values)].head()

,ORGANIZATION_TYPE_Business Entity Type 2,ORGANIZATION_TYPE_Telecom,ORGANIZATION_TYPE_Industry: type 2,OCCUPATION_TYPE_HR staff,WEEKDAY_APPR_PROCESS_START_TUESDAY,NAME_FAMILY_STATUS_Single / not married,ORGANIZATION_TYPE_Trade: type 3,ORGANIZATION_TYPE_Trade: type 1,OCCUPATION_TYPE_Sales staff,NAME_TYPE_SUITE_Unaccompanied,...,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,ORGANIZATION_TYPE_Business Entity Type 1,ORGANIZATION_TYPE_XNA,ORGANIZATION_TYPE_Industry: type 11,ORGANIZATION_TYPE_Cleaning,FLAG_OWN_REALTY_N,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_Realtor,FONDKAPREMONT_MODE_reg oper account
0,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,1,0,1
4,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0


#### Imputing

We learned earlier that a lot of features are missing values. We need to deal with the missing values before we derive the polynomial feature. 

In [14]:
type(Imputer(strategy='median'))

sklearn.preprocessing.imputation.Imputer

In [15]:
def impute(train_data: pd.DataFrame, test_data: pd.DataFrame, strategy: str) -> tuple:
    imputer = Imputer(strategy=strategy)
    
    train_imputed = imputer.fit_transform(train_data)
    train_features = pd.DataFrame(train_imputed, columns=train_data.columns)
    
    test_imputed = imputer.transform(test_data) 
    test_features = pd.DataFrame(test_imputed, columns=test_data.columns)
    
    return train_features, test_features

In [16]:
train_imputed, test_imputed = impute(train_1h, test_1h, strategy='median')

In [17]:
def count_missing_stats(data: pd.DataFrame) -> int:
    return len([(row, stat) for row, stat in (data.isnull().sum()/data.shape[0]).items() if stat>0])    

In [18]:
print ("Columns having null valies: {} (before Imputing), {} (after Imputing)"
       .format(count_missing_stats(train_1h),
               count_missing_stats(train_imputed)))

Columns having null valies: 61 (before Imputing), 0 (after Imputing)


In [19]:
print ("Columns having null valies: {} (before Imputing), {} (after Imputing)"
       .format(count_missing_stats(test_1h), 
               count_missing_stats(test_imputed)))

Columns having null valies: 58 (before Imputing), 0 (after Imputing)


### Feature Engineering

#### Polynomial feature from correlated features

In our analysis, we learned that `TARGET` is positively correlated with `DAYS_BIRTH` and negatively correlated with `EXT_SOURCE_1`, `EXT_SOURCE_2`, and `EXT_SOURCE_3`. It is worth trying out polynomial features. `scikit-learn` provides a utitlity to generate one.

In [20]:
def EngieerPolynomialFeatures(train_data: pd.DataFrame, 
                              test_data: pd.DataFrame, 
                              feature_columns: list, 
                              degree: int, 
                              merge_id: str, 
                              merge_how:str) -> tuple:
    poly_transformer = PolynomialFeatures(degree=degree)
    
    train_features = poly_transformer.fit_transform(train_data[feature_columns])
    test_features = poly_transformer.transform(test_data[feature_columns])
    
    engineered_column_names = poly_transformer.get_feature_names(feature_columns)
    
    poly_df_train = pd.DataFrame(train_features, columns=engineered_column_names)
    poly_df_train[merge_id] = train_data[merge_id]
    poly_train_features = train_data.merge(poly_df_train, how=merge_how, on=merge_id)
    
    poly_df_test = pd.DataFrame(test_features, columns=engineered_column_names)
    poly_df_test[merge_id] = test_data[merge_id]
    poly_test_features = test_data.merge(poly_df_test, how=merge_how, on=merge_id)
    
    return poly_train_features, poly_test_features

In [21]:
feature_columns = ['DAYS_BIRTH', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']
degree = 3    
merge_id = 'SK_ID_CURR'
merge_how = 'left'
poly_train_features, poly_test_features = EngieerPolynomialFeatures(train_data=train_imputed,
                                                                    test_data=test_imputed,
                                                                    feature_columns=feature_columns, 
                                                                    degree=degree, 
                                                                    merge_id=merge_id, 
                                                                    merge_how=merge_how)

In [22]:
poly_train_features.shape, poly_test_features.shape

((307511, 277), (48744, 277))

Lets look at the polynomial features

In [23]:
poly_train_features[get_extra_columns(train_imputed.columns.values, poly_train_features.columns.values)].head()

,DAYS_BIRTH EXT_SOURCE_1^2,DAYS_BIRTH_x,EXT_SOURCE_1^2,DAYS_BIRTH EXT_SOURCE_2^2,EXT_SOURCE_1 EXT_SOURCE_3^2,EXT_SOURCE_3^3,DAYS_BIRTH^3,1,DAYS_BIRTH EXT_SOURCE_3^2,EXT_SOURCE_2 EXT_SOURCE_3,...,DAYS_BIRTH^2,DAYS_BIRTH EXT_SOURCE_2 EXT_SOURCE_3,EXT_SOURCE_1^2 EXT_SOURCE_2,DAYS_BIRTH EXT_SOURCE_3,EXT_SOURCE_2 EXT_SOURCE_3^2,EXT_SOURCE_1_y,EXT_SOURCE_2_x,EXT_SOURCE_1 EXT_SOURCE_3,EXT_SOURCE_2^2,DAYS_BIRTH EXT_SOURCE_1 EXT_SOURCE_3
0,-65.234900,-9461.0,0.006895,-654.152107,0.001613,0.002707,-8.468590e+11,1.0,-183.785678,0.036649,...,89510521.0,-346.733022,0.001813,-1318.634256,0.005108,0.083037,0.262949,0.011573,0.069142,-109.495390
1,-1624.316241,-16765.0,0.096887,-6491.237078,0.089185,0.153368,-4.712058e+12,1.0,-4803.518937,0.333073,...,281065225.0,-5583.975307,0.060288,-8973.906339,0.178286,0.311267,0.622246,0.166614,0.387190,-2793.283699
2,-4876.421768,-19046.0,0.256034,-5885.942404,0.269326,0.388325,-6.908939e+12,1.0,-10137.567875,0.405575,...,362750116.0,-7724.580288,0.142332,-13895.327191,0.295894,0.505998,0.555912,0.369159,0.309038,-7031.006802
3,-4865.924377,-19005.0,0.256034,-8040.528832,0.144979,0.153368,-6.864416e+12,1.0,-5445.325225,0.348166,...,361190025.0,-6616.894625,0.166535,-10172.925140,0.186365,0.505998,0.650442,0.270849,0.423074,-5147.479068
4,-5103.267808,-19932.0,0.256034,-2076.117157,0.144979,0.153368,-7.918677e+12,1.0,-5710.929881,0.172754,...,397284624.0,-3443.335521,0.082632,-10669.126224,0.092471,0.505998,0.322738,0.270849,0.104160,-5398.555790


In [24]:
poly_test_features[get_extra_columns(test_imputed.columns.values, poly_test_features.columns.values)].head()

,DAYS_BIRTH EXT_SOURCE_1^2,DAYS_BIRTH_x,EXT_SOURCE_1^2,DAYS_BIRTH EXT_SOURCE_2^2,EXT_SOURCE_1 EXT_SOURCE_3^2,EXT_SOURCE_3^3,DAYS_BIRTH^3,1,DAYS_BIRTH EXT_SOURCE_3^2,EXT_SOURCE_2 EXT_SOURCE_3,...,DAYS_BIRTH^2,DAYS_BIRTH EXT_SOURCE_2 EXT_SOURCE_3,EXT_SOURCE_1^2 EXT_SOURCE_2,DAYS_BIRTH EXT_SOURCE_3,EXT_SOURCE_2 EXT_SOURCE_3^2,EXT_SOURCE_1_y,EXT_SOURCE_2_x,EXT_SOURCE_1 EXT_SOURCE_3,EXT_SOURCE_2^2,DAYS_BIRTH EXT_SOURCE_1 EXT_SOURCE_3
0,-10898.652144,-19241.0,0.566429,-11997.802403,0.019151,0.004059,-7.123328e+12,1.0,-489.615795,0.125965,...,370216081.0,-2423.698322,0.447283,-3069.315478,0.020094,0.752614,0.789654,0.120057,0.623554,-2310.011305
1,-5766.280398,-18064.0,0.319214,-1536.577117,0.105911,0.081161,-5.894429e+12,1.0,-3386.201665,0.126276,...,326308096.0,-2281.043619,0.093101,-7821.019554,0.054673,0.564990,0.291656,0.244619,0.085063,-4418.799416
2,-5130.407402,-20038.0,0.256034,-9812.640816,0.188894,0.228089,-8.045687e+12,1.0,-7480.393855,0.427564,...,401521444.0,-8567.521115,0.179169,-12243.044232,0.261238,0.505998,0.699787,0.309160,0.489702,-6194.955045
3,-3862.913505,-13976.0,0.276396,-3630.555667,0.197364,0.230013,-2.729912e+12,1.0,-5246.681115,0.312281,...,195328576.0,-4364.443591,0.140873,-8563.154516,0.191336,0.525734,0.509677,0.322119,0.259771,-4501.941285
4,-532.848276,-13040.0,0.040863,-2362.974127,0.057919,0.153368,-2.217342e+12,1.0,-3736.229463,0.227860,...,170041600.0,-2971.298294,0.017395,-6980.002306,0.121968,0.202145,0.425687,0.108203,0.181210,-1410.972511


See if the polynomial features have some positive or negative correlations

#### Prediction: Regular data

Scaling

In [25]:
scaler_regular = MinMaxScaler(feature_range=(0, 1))

In [64]:
train_scaled = scaler_regular.fit_transform(train_imputed)
test_scaled = scaler_regular.transform(test_imputed)

Training

In [30]:
model = LogisticRegression(C=0.0001)

In [31]:
model.fit(train_scaled, train_target)

LogisticRegression(C=0.0001, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

Prediction

In [38]:
prediction = model.predict_proba(test_scaled)

In [41]:
prediction[:,1]

array([ 0.06692558,  0.12825489,  0.08436064, ...,  0.05761464,
        0.0717017 ,  0.08816803])

In [53]:
submit = test_features[['SK_ID_CURR']]
submit = submit.assign(TARGET=prediction[:,1])

In [56]:
submit.shape

(48744, 2)

In [54]:
submit.head()

,SK_ID_CURR,TARGET
0,100001,0.066926
1,100005,0.128255
2,100013,0.084361
3,100028,0.061052
4,100038,0.125410


In [55]:
submit.to_csv('model_baseline.csv', index = False)

Scored 0.679

#### Predict: Polynomial features

Scaling

In [57]:
scaler_poly = MinMaxScaler(feature_range=(0, 1))

In [58]:
train_scaled_poly = scaler_poly.fit_transform(poly_train_features)
test_scaled_poly = scaler_poly.transform(poly_test_features)

In [59]:
model_poly = LogisticRegression(C=0.0001)

Model training

In [60]:
model_poly.fit(train_scaled_poly, train_target)

LogisticRegression(C=0.0001, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [65]:
prediction_poly = model_poly.predict_proba(test_scaled_poly)

In [68]:
submit_poly = test_features[['SK_ID_CURR']]
submit_poly = submit.assign(TARGET=prediction_poly[:,1])

In [69]:
submit_poly.to_csv('model_baseline_poly.csv', index = False)

Scored 0.722